In [5]:
import binascii
import sys
import traceback
from block import Block
from helper import read_varint, hash256
from tx import Tx
import subprocess
import os
from time import sleep
from subprocess import Popen, PIPE
import logging
import concurrent.futures
from multiprocessing import Pool


from time import sleep
from script import Script
from io import BytesIO
from helper import (
    decode_base58,
    encode_base58_checksum,
    encode_varint,
    h160_to_p2pkh_address,
    h160_to_p2sh_address,
    int_to_little_endian,
    little_endian_to_int,
    read_varint,
    sha256,
)
import segwit_addr
#logging.basicConfig(filename='parsing.log',level=logging.DEBUG,format='%(asctime)s %(message)s')

In [6]:
from neo4j import GraphDatabase

In [7]:
from logging.handlers import RotatingFileHandler

log_formatter = logging.Formatter('%(asctime)s %(levelname)s %(funcName)s(%(lineno)d) %(message)s')

logFile = 'log/parsing.log'

my_handler = RotatingFileHandler(logFile, mode='a', maxBytes=10*1024*1024,backupCount=6, encoding=None, delay=0)
my_handler.setFormatter(log_formatter)
my_handler.setLevel(logging.INFO)

app_log = logging.getLogger('root')
app_log.setLevel(logging.INFO)

app_log.addHandler(my_handler)

In [14]:
class BlockChainDB(object):

    def __init__(self, uri=None, user=None, password=None, driver=None):
        if driver is None:
            self._driver = GraphDatabase.driver(uri, auth=(user, password), encrypted=False)#REVISE LATER FOR ENCRYPTION!!!
        else:
            self._driver = driver

    def close(self):
        self._driver.close()
        
    @staticmethod    
    def _add_height(tx, block_id):
    
        for i in range(10000):
            result = tx.run("MATCH (b:block {id:$block_id})-[:LINKS]->(n:block)-[:LINKS]->(:block)"
                            "-[:LINKS]->(:block)-[:LINKS]->(:block) "
                            "WITH b.height AS height, n AS next_block "
                            "SET next_block.height = height+1 "
                            "RETURN next_block.id ",
                            block_id=block_id)
            
                
            block_id = result.single().value()
            print(block_id)
        return block_id
    
    @staticmethod    
    def _config_constrains(tx ):
    
        result1 = tx.run("MERGE (b:block {id:'0000000000000000000000000000000000000000000000000000000000000000', height:-1}) "
                        "MERGE (t:transaction {id:'CONFIG'}) "
                         "MERGE (a:address {address:'CONFIG'}) "
                        "RETURN b,t,a ")
        
        result = tx.run("CREATE CONSTRAINT ON (t:transaction) ASSERT t.id IS UNIQUE "
                        "CREATE CONSTRAINT ON (a:address) ASSERT a.address IS UNIQUE "
                        "CREATE CONSTRAINT ON (b:block) ASSERT b.id IS UNIQUE "
                        "CREATE CONSTRAINT ON (b:block) ASSERT b.height IS UNIQUE ")

        
        return result1
    
    @staticmethod    
    def _create_addresses(tx,batch,testnet=True):
        """
        testnet is TRUE always. Change this for mainnet later.
        """
        print(f" batch {batch},",end='')
        for height in range(batch*10,batch*10+10):

            result = tx.run("MATCH (b:block {height:$height}) "
                             "MATCH (x)<-[:CREATES]-(t:transaction)<-[:CONTAINS]-(b) "
                             "RETURN x.script_pubkey, x.index, t.id", height=height)

            addresses = []
            for output in result.data():
                address = None
                addr_type = None
                raw_script_pubkey = output["x.script_pubkey"]
                b_sp = bytes.fromhex(raw_script_pubkey)
                length = encode_varint(len(b_sp))
                stream = BytesIO(length+b_sp)
                
                try: 
                    script_pubkey = Script.parse(stream)
                
                    if script_pubkey.is_p2pkh_script_pubkey(): 
                        address= h160_to_p2pkh_address(script_pubkey.cmds[2], testnet)
                        addr_type = "P2PKH"
                    elif script_pubkey.is_p2sh_script_pubkey():  
                        address= h160_to_p2sh_address(script_pubkey.cmds[1], testnet)
                        addr_type = "P2SH"
                    elif script_pubkey.is_p2wpkh_script_pubkey() or script_pubkey.is_p2wsh_script_pubkey(): 
                        if testnet: address = segwit_addr.encode("tb",0,script_pubkey.cmds[1])
                        else: address = segwit_addr.encode("bc",0,script_pubkey.cmds[1]) 
                        if script_pubkey.is_p2wpkh_script_pubkey(): addr_type = "P2WPKH"
                        else: addr_type = "P2WSH"
                    elif len(script_pubkey.cmds)==2 and script_pubkey.cmds[1]==0xac:
                        try: 
                            address = script_pubkey.cmds[0].hex()
                            addr_type = "P2PK"
                        except: print(f"P2PK failed {script_pubkey.cmds[0]} from tx: {output['t.id']}")

                except: print(f"script parsing failed in tx {output['t.id']} index {output['x.index']} ",end="")
                    
                if address is not None:
                    address_dict = {
                        "address":address,
                        "type": addr_type,
                        "tx_id":output["t.id"],
                        "index":output["x.index"]
                    }
                    addresses.append(address_dict)

            if len(addresses)>0:
                result = tx.run("FOREACH (address in $addresses | \n"
                                "MERGE (o:output {index:address.index})<-[:CREATES]-(:transaction {id:address.tx_id}) \n"
                                "MERGE (a:address {address:address.address}) SET a.type=address.type \n"
                                "MERGE (a)-[:HAS]->(o) )", addresses=addresses)

        
        return
    def create_address_batch(self,batch):
        with self._driver.session() as session:
            result = session.write_transaction(self._create_addresses,batch)
            #print(result)
            return result
        
            
        
    def add_height_batch(self,block_id=None):
        with self._driver.session() as session:
            result = session.write_transaction(self._add_height, block_id)
            #print(result)
            return result
        
    def create_addresses(self):
        """
        modify the range of the for loop to fit with the length of the blockchain and the begginning of 
        the desired operation.
        """
        for batch in range(89365,175000):
            self.create_address_batch(batch)
            #sleep(0.2)
        
    def add_height(self, block_id=None):
        #If no specific block to start from, then we start with the genesis block.
        if block_id is None: block_id="000000000933ea01ad0ee984209779baaec3ced90fa3f408719526f8d77f4943"
            
        for i in range(2000):
            print(i)
            block_id = db.add_height_batch(block_id)
            print(block_id)
            sleep(1)
        
    def config_constrains(self):
        with self._driver.session() as session:
            result = session.write_transaction(self._config_constrains)
            #print(result)
            return result
        
    @staticmethod
    def _new_address(tx, address,i,change_addr):
        if change_addr: addr_type = "change"
        else: addr_type = "recipient"
        result = tx.run("CREATE a = (:address {address:$address, acc_index:$index, type:$kind, created:timestamp()}) "
                        "RETURN a ", address=address, index = i, kind = addr_type)
        return result.single()
    
    
    @staticmethod
    def _new_tx(tx,block_id, version, locktime, tx_id, inputs, outputs, segwit, i):
                    
        result = tx.run("MATCH (b:block {id:$block_id}) "
                        "MERGE (t:transaction {id:$tx_id}) "
                        "SET t.version=$version, t.segwit=$segwit, t.locktime=$locktime "
                        "MERGE (t)<-[:CONTAINS {i:$i}]-(b) "
                        "RETURN t ",
                        tx_id=tx_id, segwit=segwit, version=version, locktime=locktime, block_id=block_id,i=i)
        
        if len(result.data()) >0: app_log.info(f"created tx: {tx_id}\n{result.data()}")
        else: app_log.error(f"FAILED AT CREATING TX {tx_id}")
        
        for index,tx_out in enumerate(outputs):
            result = tx.run("MATCH (t:transaction {id : $tx_id}) "
                        "MERGE (o:output {index:$index})<-[:CREATES]-(t) "
                        "SET o.amount=$amount, o.script_pubkey=$script_pubkey "
                        "RETURN o ", 
                        tx_id = tx_id,amount=tx_out.amount,script_pubkey=tx_out.script_pubkey.hex(), index=index)
            if len(result.data()) > 0: app_log.info(f"created output: {tx_out.amount}:{index} for tx {tx_id}")
            else: app_log.error(f"FAILED AT CREATING OUTPUT {index}")  
                
        for tx_in in inputs:
            if tx_in.prev_index == 4294967295 or tx_in.prev_tx == (b'\x00'*32).hex():
                    app_log.info("Coinbase Transaction " + tx_id)
            else: 
                result = tx.run("MERGE (t:transaction {id : $tx_id}) "
                            #"MERGE (p:transaction {id:$prev_tx})"
                            "MERGE (tx_in :output {index:$prev_index})<-[:CREATES]-(:transaction {id:$prev_tx}) "
                            "MERGE (tx_in)<-[r:SPENDS {script_sig:$script_sig, witness:$witness}]-(t) "
                            "RETURN r",
                            tx_id=tx_id, prev_index=tx_in.prev_index, prev_tx=tx_in.prev_tx.hex(), 
                            script_sig=tx_in.script_sig.hex(), witness=tx_in.witness.hex() )
                if len(result.data()) > 0: app_log.info(f"created input for transaction {tx_id} spending from {tx_in.prev_tx.hex()} index {tx_in.prev_index}")
                else: app_log.error(f"FAILED AT CREATING INPUT {tx_in.prev_tx}:{tx_in.prev_index}")  

        return True
    
    @staticmethod
    def _new_block(tx,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx):
        #_height = tx.run( "MATCH (u:block) RETURN COUNT (u) ").single()[0] 
        #pblock = tx.run("MATCH (prev_block:block {id:$prev_block}) RETURN prev_block", prev_block=prev_block)
        #if not pblock.single():
         #   print(f"COULD NOT FIND PREVIOUS BLOCK {prev_block}.")
        result = tx.run("MERGE (block:block {id:$block_id}) "
                        "SET block.n_tx=$n_tx, block.nonce=$nonce, block.merkle_root=$merkle_root, block.bits=$bits, block.timestamp=$timestamp, block.version=$version "
                        "MERGE (prevblock:block {id:$prev_block}) "
                        "MERGE (block)<-[:chain]-(prevblock) "
                        "RETURN block ",
                        block_id=block_id, version=version, prev_block=prev_block, merkle_root=merkle_root, timestamp=timestamp, 
                        bits=bits, nonce=nonce, n_tx=n_tx)
        return result.data()

    def new_address(self, address,i,change_addr):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_address, address,i,change_addr)
            print(result)
            
    def new_tx(self, block_id, version, locktime, tx_id, inputs, outputs, segwit,i):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_tx, block_id, version, locktime, tx_id, inputs, outputs, segwit,i)
                  
    def new_block(self,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_block,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx)
            if len(result) >0: app_log.info(f"CREATED BLOCK {block_id}")
            else: app_log.error(f"FAILED AT CREATING BLOCK {block_id}")
            #print(result)
  
      

In [1]:

def manager(args):
    """
    args is a touple of 2 arguments: (arg1, arg2)
    arg1 is the index of the first blk#####.dat file that the parser will work on.
    arg2 is the number of threats, and therefore the number of files the parser will work on.
    For example, if the arguments are (2,2), this means that the parser will work on files
    blk00002.dat and blk00003.dat at the same time.
    arg2 should never be more than 3 for efficiency reasons.
    """
    #n_threads should be 3 or 2 to get maximum efficiency.
    n_threads = args[1]
    n = args[0]
    #db = BlockChainDB(driver = args[2])
    db = BlockChainDB("neo4j://localhost:7687", "neo4j", "wallet")
    file_list = [(f"{i:05}",db) for i in range( n , n + n_threads )]
    with concurrent.futures.ThreadPoolExecutor(max_workers=n_threads) as executor:
        executor.map(parse_blockchain, file_list)   
    return True    

def get_cursor(file):
    c = 0
    print(f"checking for cursors {file}")
    try:
        cursor = open(f"cursors/cursor{file}.txt")
        c = cursor.readline()
        try:
            c = int(c)
            print(f"cursor at {c} for file blk{file}.dat")
            coursor.close()
            return c
        except:
            if c == "finished":
                print(c)
                return True
            else:
                try: 
                    print(f"trying to recover file from backup for file blk{file}.dat.")
                    cursor = open(f"{file[:-4]}.txt.bck")
                    c = coursor.readline()
                    try:
                        c = int(c)
                        subprocess.call(f" cp cursors/cursor{file}.txt.bck cursors/cursor{file}.txt", shell=True)
                        print(f"succesfully recovered file from backup for file blk{file}.dat. Restored original file.")
                        coursor.close()
                        return c
                    except:
                        if c == "finished":
                            coursor.close()
                            print(c)
                            subprocess.call(f" cp cursors/cursor{file}.txt.bck cursors/cursor{file}.txt",shell=True)
                            print(f"Finished. Succesfully updated original file for file blk{file}.dat.")
                            return c
                        else:
                            print(f"Corrupted cursor files for file blk{file}.dat.")
                            raise Exception

                except:
                    print(f"No back-up file for file blk{file}.dat.")
                    raise Exception


    except:
        cursor  = open(f"cursors/cursor{file}.txt","w")
        print(f"No cursor file for file blk{file}.dat.")
        cursor.write(str(c))
        print(c)
        cursor.close()
        return c

def parse_blockchain(args):
    """
    args: ( file index (######),  database driver )
    """
    file = f"blocks_demo/blk{args[0]}.dat"
    db=args[1]
    print(f"parsing {file}")
    #print(f"driver {db}")

    with open(file,"rb") as block_file:
        print(f"opened {file}")
        c = get_cursor(args[0])
        if c !=0: 
            print(f"reading from {c} for file {file}.")
            block_file.read(c)

        #infinite loop to parse the blk#####.dat file. 
        #Only stops when an  error occures or the file is over.
        while True:

            try:
                #parse the block using the class Block from the file block.py
                this_block = Block.parse_from_blk(block_file)

                #calculate the block id using the header of the parsed block:
                #first the header is concatenated
                header = this_block.version.to_bytes(4,"little")+this_block.prev_block[::-1]+this_block.merkle_root[::-1]+this_block.timestamp.to_bytes(4,"little") + this_block.bits + this_block.nonce
                #then the concatenated header is hashed to get the block id.
                block_id = hash256(header)[::-1]

                #A new block is created in the database using the parsed block and its id.
                db.new_block(block_id.hex(),this_block.version, this_block.prev_block.hex(), 
                             this_block.merkle_root.hex(),this_block.timestamp, 
                             int.from_bytes(this_block.bits,"big"), 
                             int.from_bytes(this_block.nonce,"big"),
                             this_block.tx_hashes)

                #Every transaction in the block is parsed using the Tx class from the file tx.py
                for transaction in range(this_block.tx_hashes):
                    #the current transaction is parsed 
                    tx = Tx.parse(block_file)
                    #the parsed transaction is created in the database 
                    db.new_tx(block_id.hex(), tx.version, tx.locktime, tx.id(), tx.tx_ins, tx.tx_outs, tx.segwit,transaction)

                #the cursor is updated to start reading the next block. 8 bytes are added since some info comes before the flag. 
                c += (this_block.size + 8)
                #print(c)
                #the cursor file and its backup file are updated
                cursor  = open(f"cursors/cursor{args[0]}.txt","w")
                cursor.write(str(c))
                cursor.close()
                cursor  = open(f"cursors/cursor{args[0]}.txt.bck","w")
                cursor.write(str(c))
                cursor.close()


            except Exception as e:
                print(f"Finished {file} file. ")
                print(e)
                print(e.with_traceback)
                break





 

In [13]:

        
def super_main(start_at,n_processes,n_threads):
    #driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "wallet"), encrypted=False)
    batch_list = [start_at+n_threads*i for i in range(n_processes)]
    arg_list = [(n,n_threads) for n in batch_list]
    with Pool(n_processes) as p:
        p.map(main, arg_list)
    
def manager(total_files, n_processes, n_threads, n_nodes):
    concurrent_files = n_processes*n_threads
    batches = total_files//concurrent_files+1
    batch_list = [concurrent_files*i for i in range(batches)]
    print(f"batch_list {batch_list}")
    for index,batch in enumerate(batch_list):
        node = index%n_nodes
        print(f"node {node} batch {batch}")
        super_main(batch, n_processes, n_threads)

In [1]:
import parser 

In [ ]:
parser.manager((2,2))

parsing blocks_demo/blk00002.dat
parsing blocks_demo/blk00003.dat
opened blocks_demo/blk00002.dat
checking for cursors 00002
opened blocks_demo/blk00003.dat
checking for cursors 00003
cursor at 202952 for file blk00002.dat
trying to recover file from backup for file blk00002.dat.
cursor at 169259 for file blk00003.dat
trying to recover file from backup for file blk00003.dat.
No back-up file for file blk00002.dat.No back-up file for file blk00003.dat.

No cursor file for file blk00003.dat.
169259
No cursor file for file blk00002.dat.
202952
reading from 169259 for file blocks_demo/blk00003.dat.
reading from 202952 for file blocks_demo/blk00002.dat.


In [20]:
manager(10, 4,2,1)

batch_list [0, 8]
node 0 batch 0
parsing blocks_demo/blk00006.dat
parsing blocks_demo/blk00004.dat
parsing blocks_demo/blk00005.dat
parsing blocks_demo/blk00000.dat
parsing blocks_demo/blk00001.dat
parsing blocks_demo/blk00007.dat
opened blocks_demo/blk00000.dat
opened blocks_demo/blk00004.dat
opened blocks_demo/blk00007.dat
opened blocks_demo/blk00005.dat
checking for cursors 00000
checking for cursors 00004
opened blocks_demo/blk00006.dat
opened blocks_demo/blk00001.dat
checking for cursors 00005
checking for cursors 00006
parsing blocks_demo/blk00002.dat
cursor at 27417332
checking for cursors 00007
cursor at 75195
checking for cursors 00001
parsing blocks_demo/blk00003.dat
opened blocks_demo/blk00002.dat
cursor at 66491
cursor at 43942
trying to recover file from backup.
opened blocks_demo/blk00003.dat
cursor at 70919
trying to recover file from backup.
cursor at 25418670
checking for cursors 00002
trying to recover file from backup.
checking for cursors 00003
trying to recover fil

Process ForkPoolWorker-19:
Process ForkPoolWorker-18:
Process ForkPoolWorker-20:
Process ForkPoolWorker-17:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self

KeyboardInterrupt: 

In [ ]:
main()

parsing blocks_demo/blk00000.dat
parsing blocks_demo/blk00001.dat
opened blocks_demo/blk00000.dat
checking for cursors 00000
parsing blocks_demo/blk00002.dat
opened blocks_demo/blk00001.dat
checking for cursors 00001
No cursor file
0
No cursor fileopened blocks_demo/blk00002.dat
checking for cursors 00002

0
No cursor file
0


# The Idea For The RPi Cluster

Since python can only manage one process, we would have to manage the script from outside python. In other words, we will have to call the python script 4 times from a bash script in order for them to start in an independent process, and in this way utilize the 4 cores of the raspberry pi. Each one of the four scripts will be assigned a different blk\*\*\*\*\*.dat file to parallelize the parsing. If this works, this will mean that the cluster will be able to parse 16 files at the same time.

To Do/Fix:
1. Skip the Script op_codes parsing on the ScriptPubKey and ScripSig. This will be done separetely if necessary, but for efficiency reasons, the database will store these fields as hexadecimals without any validation since this is also unnecessary. This will fix the app crashing on ScriptSig for coinbase transactions where the scripts are abitrary.
2. Create the bash script that will assign a different file to the python script.
3. Create the file sharing system for the blocks and the database.
4. 

In [32]:
os.chdir("/Users/oscareduardosernarosero/Desktop/neo4j-community-4.0.3/bin")
whereami = subprocess.check_output("pwd")
print(whereami)
if whereami == b"/Users/oscareduardosernarosero/Desktop/neo4j-community-4.0.3/bin\n": print("ok")
print(subprocess.check_output("ls"))
start_neo4j = subprocess.check_output("./neo4j start",shell = True)
if "Started neo4j" in str(start_neo4j): print("Sarted Neo4j successfully...")
else: print("Failed starting Neo4j.")
sleep(2)

In [57]:
#os.chdir("/Users/oscareduardosernarosero/Desktop/neo4j-community-4.0.3/bin")
#output = Popen(["./cypher-shell","-u","neo4j","-p","wallet"],stdout=PIPE, stderr=PIPE)
subprocess.call("./cypher-shell -u neo4j -p wallet",shell = True)
output = subprocess.check_output("MATCH (n) RETURN n",shell = True)
print(output)

CalledProcessError: Command 'MATCH (n) RETURN n' returned non-zero exit status 2.

In [54]:
stdout, stderr = output.communicate("\tMATCH (n) RETURN n\n")
print (stdout)
#subprocess.call("neo4j",shell = True)
#sleep(2)
#loggedin = subprocess.check_output("wallet",shell = True)
#if "Connected to Neo4j 4.0.3 at neo4j://localhost:7687 as user neo4j" in str(loggedin):print("connected")

b''


In [13]:
with open("blocks_demo/blk00000.dat","rb") as block_file:
    
    print(block_file.read(297))
    

b"\xf9\xbe\xb4\xd9\x1d\x01\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00;\xa3\xed\xfdz{\x12\xb2z\xc7,>gv\x8fa\x7f\xc8\x1b\xc3\x88\x8aQ2:\x9f\xb8\xaaK\x1e^J)\xab_I\xff\xff\x00\x1d\x1d\xac+|\x01\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xffM\x04\xff\xff\x00\x1d\x01\x04EThe Times 03/Jan/2009 Chancellor on brink of second bailout for banks\xff\xff\xff\xff\x01\x00\xf2\x05*\x01\x00\x00\x00CA\x04g\x8a\xfd\xb0\xfeUH'\x19g\xf1\xa6q0\xb7\x10\\\xd6\xa8(\xe09\t\xa6yb\xe0\xea\x1fa\xde\xb6I\xf6\xbc?L\xef8\xc4\xf3U\x04\xe5\x1e\xc1\x12\xde\\8M\xf7\xba\x0b\x8dW\x8aLp+k\xf1\x1d_\xac\x00\x00\x00\x00\xf9\xbe\xb4\xd9"


In [34]:
hex(20)

'0x14'

In [ ]:
os.chdir("block_demo")
files = subprocess.check_output("ls")
files

In [13]:
with open("blocks_demo/blk00000.dat","rb") as block_file:
    #block_file.read(43942)
    
    this_block = Block.parse_from_blk(block_file)

    header = this_block.version.to_bytes(4,"little")+this_block.prev_block[::-1]+this_block.merkle_root[::-1]+this_block.timestamp.to_bytes(4,"little") + this_block.bits + this_block.nonce
    block_id = hash256(header)[::-1]
    print(block_id.hex())

    print(block_id.hex(),this_block.version, this_block.prev_block.hex(), 
                 this_block.merkle_root.hex(),this_block.timestamp, 
                 int.from_bytes(this_block.bits,"big"), 
                 int.from_bytes(this_block.nonce,"big"),
                 this_block.tx_hashes)
   
    for transaction in range(this_block.tx_hashes):
        tx = Tx.parse(block_file)
        print(tx)
        print(tx.tx_ins[0].script_sig.cmds[2])
        #db.new_tx(block_id.hex(), tx.version, tx.locktime, tx.id(), tx.tx_ins, tx.tx_outs, tx.segwit,transaction)


000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f
000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f 1 0000000000000000000000000000000000000000000000000000000000000000 4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b 1231006505 4294901789 497822588 1
tx: 4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b
version: 1
tx_ins:
0000000000000000000000000000000000000000000000000000000000000000:4294967295
tx_outs:
5000000000:04678afdb0fe5548271967f1a67130b7105cd6a828e03909a67962e0ea1f61deb649f6bc3f4cef38c4f35504e51ec112de5c384df7ba0b8d578a4c702b6bf11d5f OP_CHECKSIG
locktime: 0
b'The Times 03/Jan/2009 Chancellor on brink of second bailout for banks'


In [9]:
def add_height():
    
    tx_id="000000000933ea01ad0ee984209779baaec3ced90fa3f408719526f8d77f4943"
    #while (True):      
    tx_id = tx.run("MATCH (t:transaction {id:$tx_id})-[:LINKS]->(n) "
                    "WITH t.height AS height, n AS next_block"
                    "SET next_block.height = height+1"
                    "RETURN next_block.id",
                    tx_id=tx_id)
    print(tx_id)



In [15]:
from neo4j import GraphDatabase
db = BlockChainDB("neo4j://localhost:7687", "neo4j", "wallet")
#r = db.add_height()

In [35]:
db.add_height("0000000000000a8813a0f9ef03bac3a0af5eaea8e19cc236bef9f6b3d1613ab7")

0
0000000000000abc0fe6ca86b65cdede2de0213e7e28f302f85e2cb7ab5fd7ba
00000000000000b39d052fa7b026d29fbac601fdbaec54375151f5fb0e5bbddf
0000000000000f5133e6ebdb8fab60736f380eaaec9079d952177bf26871db9b
00000000000008b3d2a76647934a0ba5ff77aaf8c21175b99eeaf7413a7b20e8
0000000000000139af2a38f3150aefb6ca6b21ce11c7cce0cded9185581862c5


AttributeError: 'NoneType' object has no attribute 'value'

## Wierd blocks

000000000003c97dd492e3ff63701e67281e59e2833c6fd560573f2765a08344

0000000000032e60bc90a99b204a0090a249ec5e5905ce1f6f1a63e9f551cb75

000000000000529b722600ddf20e51beee21498185da0dcea5b6759c45534c43

0000000000005af500049df0a49ab6385a6b1474da48c8112c4a3d21cabacb38

00000000000137c2eef9babdb62770a46fafb5f3972fffae96f576b79e1a9ff8

000000000000f6ad4f0dcace87dd62a844fb75e62ef7d3f037a82fe0082ef72d

000000000000273b6885ec863a47fe86cbeaf23d62efaf19cb9fcc8210afc013

0000000000000cbf70ec51aa3810b837cf8e3edf01bfcc0ee28b12d4f24d96c2

In [47]:

b_sp = bytes.fromhex("a9149a9f39b1ee13e3d324b18cd389d633a54dcfd5f587")
length = encode_varint(len(b_sp))
stream = BytesIO(length+b_sp)
x = Script.parse(stream)

In [48]:
x.cmds

[169,
 b'\x9a\x9f9\xb1\xee\x13\xe3\xd3$\xb1\x8c\xd3\x89\xd63\xa5M\xcf\xd5\xf5',
 135]

In [16]:
db.create_addresses()

 batch 89365,

KeyboardInterrupt: 

In [ ]:
MATCH (b:block {height:893651})-[:COINBASE]->(:transaction)-[:CREATES]->(:output)<-[:SPENDS]-(t:transaction) RETURN t